# 직렬화 및 프로토콜 버퍼

In [1]:
import tensorflow as tf
import numpy as np
import os 
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = '/tmp/data'
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

NUM_STEPS = 1000
MINIBATCH_SIZE = 100

DIR = 'saved_model2/'

x = tf.placeholder(tf.float32, [None, 784],name='x')
W = tf.Variable(tf.zeros([784, 10]),name='W')
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)
cross_entropy = tf.reduce_mean(
             tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

saver = tf.train.Saver(max_to_keep=7, 
                       keep_checkpoint_every_n_hours=1)

train_var = [x,y_true,accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1,NUM_STEPS+1):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})
        
        if step % 50 == 0:
            saver.export_meta_graph(os.path.join(DIR,"model_ckpt.meta"),collection_list=['train_var'])
            saver.save(sess, os.path.join(DIR, "model_ckpt"), 
                                           global_step=step)
    
    ans = sess.run(accuracy, feed_dict={x: data.test.images, 
                                        y_true: data.test.labels})

print("Accuracy: {:.4}%".format(ans*100))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Accuracy: 91.72%


# Export

In [2]:
tf.reset_default_graph() 

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.import_meta_graph(os.path.join(DIR,"model_ckpt.meta"))
    saver.restore(sess,  os.path.join(DIR,"model_ckpt-1000"))
    x =  tf.get_collection('train_var')[0]
    y_true =  tf.get_collection('train_var')[1]
    accuracy =  tf.get_collection('train_var')[2]

    ans = sess.run(accuracy, feed_dict={x: data.test.images, 
                                        y_true: data.test.labels})
print("Accuracy: {:.4}%".format(ans*100))

INFO:tensorflow:Restoring parameters from saved_model2/model_ckpt-1000
Accuracy: 91.72%
